### Set path interface. Please change to your own path

In [1]:
import sys
sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANI

In [2]:
import numpy as np
import time
# ASE
import  ase
from ase.io import read, write
from ase.optimize import BFGS, LBFGS 
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

#figure plotting
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

Read geometry from xyz file

In [3]:
geometry = read('data/water.xyz')

Setup ANI and calculate single point energy

In [4]:
geometry.set_calculator(ANI())
e = geometry.get_potential_energy()
print('Total energy', e, 'eV')

Total energy -2078.63121157 eV


In [5]:
geometry.get_forces()

array([[ 0.19142392, -0.2092285 ,  0.00468441],
       [-0.0934471 ,  0.23035382, -0.00543961],
       [-0.09797663, -0.02112528,  0.00075519]], dtype=float32)

Geometry optimization with BFGS

In [6]:
start_time = time.time()
dyn = LBFGS(geometry)
dyn.run(fmax=0.001)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

       Step     Time          Energy          fmax
LBFGS:    0 16:21:56    -2078.631212        0.2836
LBFGS:    1 16:21:56    -2078.631610        0.1856
LBFGS:    2 16:21:56    -2078.631885        0.0167
LBFGS:    3 16:21:56    -2078.631890        0.0091
LBFGS:    4 16:21:56    -2078.631892        0.0035
LBFGS:    5 16:21:56    -2078.631894        0.0003
[ANI Total time: 0.017764806747436523 seconds]


In [7]:
e = geometry.get_potential_energy()
print('Total energy', e, 'eV')

Total energy -2078.63189359 eV


In [8]:
geometry.get_forces()

array([[ -2.30617457e-06,  -2.97927356e-04,   7.32954868e-06],
       [ -6.46489134e-05,   2.63106631e-04,  -6.31980538e-06],
       [  6.72085152e-05,   3.45736116e-05,  -1.01132730e-06]], dtype=float32)

Vibrations and thermochemistry with numeric differentiation

In [9]:
vib = Vibrations(geometry, nfree=2) 
vib.run()

Writing vib.eq.pckl
Writing vib.0x-.pckl
Writing vib.0x+.pckl
Writing vib.0y-.pckl
Writing vib.0y+.pckl
Writing vib.0z-.pckl
Writing vib.0z+.pckl
Writing vib.1x-.pckl
Writing vib.1x+.pckl
Writing vib.1y-.pckl
Writing vib.1y+.pckl
Writing vib.1z-.pckl
Writing vib.1z+.pckl
Writing vib.2x-.pckl
Writing vib.2x+.pckl
Writing vib.2y-.pckl
Writing vib.2y+.pckl
Writing vib.2z-.pckl
Writing vib.2z+.pckl


In [10]:
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    3.0i     24.2i
  1    0.2i      1.6i
  2    0.1i      0.7i
  3    1.2      10.0 
  4    3.2      25.9 
  5    3.5      28.4 
  6  263.6    2126.2 
  7  462.9    3733.8 
  8  481.4    3882.4 
---------------------
Zero-point energy: 0.608 eV


In [11]:
vib.get_zero_point_energy()

0.60794458439021981

In [12]:
vib.get_frequencies()

array([    0.00000000+24.18860311j,     0.00000000 +1.55378813j,
           0.00000000 +0.69435977j,    10.04881929 +0.j        ,
          25.86708252 +0.j        ,    28.36312994 +0.j        ,
        2126.23944136 +0.j        ,  3733.84538185 +0.j        ,
        3882.44374080 +0.j        ])

In [13]:
vib_energies = vib.get_energies()

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=e,
                        atoms=geometry,
                        geometry='nonlinear',
                        symmetrynumber=1, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)

Enthalpy components at T = 298.15 K:
E_pot              -2078.632 eV
E_ZPE                  0.604 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.000 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                  -2077.925 eV

Entropy components at T = 298.15 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 atm)    0.0015008 eV/K        0.447 eV
S_rot              0.0005145 eV/K        0.153 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0000000 eV/K        0.000 eV
S (1 atm -> P)    -0.0000000 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0020153 eV/K        0.601 eV

Free energy components at T = 298.15 K and P = 101325.0 Pa:
    H      -2077.925 eV
 -T*S         -0.601 eV
-----------------------
    G      -2078.526 eV


In [14]:
G

-2078.5260237005964

In [15]:
# remove mode pckl files
vib.clean()

19